# SQL 고급 활용 및 튜닝

# 제 1장 아키텍처 기반 튜닝 원리

# 제1절 데이터베이스 아키텍처

## 1. 아키텍처 개관

### 가. ORACLE 아키텍처

- 데이터베이스 : 물리적인 디스크에 저장된 데이터집합 (데이터파일, 리두로그파일, 컨트롤파일)
- 인스턴스 : 공유메모리(SGA)와 이를 엑세스하는 프로세스 집합

#### 하나의 인스턴스는 하나의 데이터베이스를 액세스(Single)
#### 여러개의 인스턴스는 하나의 데이터베이스를 엑세스(RAC)

### 나. SQL Server 아키텍처

- 하나의 인스턴스당 최고 32,767개의 데이터베이스를 정의해서 사용
- 기본적으로 시스템데이터베이스가 만들어지면, 사용자데이터베이스를 추가하여 생성하는 구조

- 시스템데이터베이스 : mster, model, msdb, tempdb 등
- 사용자데이터베이스 : 데이터파일(mdf), 트랜잭션로그파일(ldf), 보조데이터파일(ndf)

## 2. 프로세스

- 서버프로세스 : 전면에 나서 사용자가 던지는 각종 명령을 처리
- 백그라운드프로세스 : 뒤에서 묵묵히 주어진 역할을 수행

### 가. 서버프로세스(Server Process)

- 사용자 프로세스와 통신하면서 사용자의 각종 명령어 처리
- ORACLE : 서버프로세스
- SQL Server : Worker thread

#### 처리절차
- 사용자의 요청
- SQL 파싱
- 커서를 열어서 SQL을 실행하면서 블록 READ
- 읽은 데이터를 정렬하여 요청한 결과집합을 만들어 네트워크를 통해 전송

#### 클라이언트가 서버프로세스와 연결하는 방식(예 오라클)

#### 전용서버 방식(Dedicated Server)
- 클라이언트 세션과 서버프로세스가 1:1 로 매핑
- 오라클의 가장 일반적인 방식
- 클라이언트 요청에 의해 리스너 프로세스는 dedicated server 생성
- 새로운 dedicated server 프로세스는 리스너에 의해 커넥션 권한을 상속받음
- 데이터베이스와 물리적인 커넥션을 맺음


1. (User)     연결요청                       ==> (Listener)
2. (Listnenr) 프로세스 생성 및 연결요청 상속 ==> (Server)
3. (Server)   RESEND 패킷 전송               ==> (User)
4. (User)     연결 후 작업 요청              ==> (Server)
5. (Server)   처리 후 결과 전송              ==> (User)
 

#### 공유서버 방식(Shared Server)
- 클라이언트 세션과 서버프로세스가 1:N로 매핑
- 클라이언트 요청에 의해 리스너 프로세스는 현재 사용가능한 dispatcher pool 탐색확인
- 리스너는 사용가능한 dispatcher 커넥션 정보를 클라이언트에 되돌려줌
- 클라이언트는 리스너 접속을 끝내고 바로 dispatcher로 접속

1. (User) 연결요청 ==> (Listener)
2. (Listener) 가용한 Dispatcher 포트번호 전송 ==> (User)
3. (User) 연결 후 작업 요청 ==> (Dispatcher)
4. (Dispatcher) 요청등록 ==> (SGA Request Queue)
5. (SGA Request Queue) 요청접수 ==> (Server)
6. (Server) 결과 등록 ==> (SGA Reponcse Queue)
7. (SGA Reponcse Queue) 결과수렴 ==> (Dispatcher)
8. (Dispatcher) 결과 전송 ==> (User)

### 나. 백그라운드프로세스

####              오라클                 ==                       SQL SERVER
**SMON(System Monitor) == Databese cleanup / Shrinking Thread**
- 장애가 발생한 시스템을 재기동할 때 인스턴스 복구를 수행하고, 임시 세그먼트와 익스텐트를 모니터링한다.

**PMON(Process Monitor) == Open data Services(OPS)**
- 이상이 생긴 프로세스가 사용하던 리소스를 복구한다.

**DBWn(Database Writers) == Lazywriter Thread**
- 버퍼 캐시에 있는 더티 버퍼를 데이터 파일에 기록

**LGWR(Log Writer) == Log writer Thread**
- 로그 버퍼 엔트리를 redo 로그 파일에 기록한다.

**ARCn(Archiver) == N/A**
- 꽉찬 리두로그가 덮어 쓰여지기 전에 archive로그 디렉토리로 백업한다.

**CKPT(Checkpoint) == Database Checkpoint Thread**
- checkpoint 프로세스는 이전의 checkpoint가 일어났던 마지막 시점 이후의 데이터베이스 변경 사항을 데이터파일에 기록하도록  
트리거링하고, 기록이 완료되면 현재 어디까지 기록했는지를 컨트롤 파일과 데이터 파일 헤더에 기록한다.
- 좀더 자세히 설명하면 write Ahead Logging 방식을 사용하는 DBMS는 리두로그에 기록해 둔 버퍼 브록에 대한 변경사항 중
현재 어디까지를 데이터 파일에 기록했는지 체크 포인트정보를 관리해야 한다.
- 이는 버퍼캐시와 데이터 파일이 동기화된 시점을 가리키며, 장애가 발생하면 마지막 체크포인트 이후 로그 데이터만 디스크에 
기록함으로써 인스턴스를 복구할 수 있도록 하는 용도로 사용된다.
- 이 정보를 갱신하는 주기가 길수록 장애 발생시 인스턴스 복구 시간도 길어진다.

**RECO(Recoverer) == Distributed Transaction Coordinator(DTC)**
- 분산 트랜잭션 과정에 발생한 문제를 해결한다.

## 3. 파일구조

### 가. 데이터파일

1) **블록(=페이지)**
- 대부분의 DBMS에서는 I/O 블록단위로 이루어짐
- 데이터를 읽고 쓸때의 논리적인 단위
- SQL 성능을 좌우하는 가장 중요한 성능지표
- 옵티마이저의 파단에 가장 큰 영향을 미치는 요소

2) **익스텐트(Extent)**
- 테이블스페이스로부터 공간을 할당하는 단위

3) **세그먼트(Segment)**
- 테이블, 인덱스, Undo 처럼 저장공간을 필요로하는 데이터베이스 오브젝트 (한개 이상의 익스텐트 사용)
- 파티션은 오브젝트와 세그먼트가 1:M (파티션을 만들면 내부적으로 여러개의 세그먼트가 만들어짐)
- 한 세그먼트에 할당된 엑스텐트가 여러 데이터파일에 흩어져 저장됨 (디스크 경합감소. I/O 분산효과)

4) **테이블스페이스(Tablespace)**
- 세그먼트를 담는 콘테이너로서 여러개의 데이터파일로 구성됨
- 사용자는 데이터파일을 직접 선택할 수 없으므로 실제 파일을 선택하고 익스텐트를 할당하는것은 DBMS의 몫

### 나. 임시파일

- 대량의 정렬이나 해시 작업을 수행하다가 메모리 공간이 부족해지면 중간 결과집합을 저장하는 용도
- 오라클에서는 임시 테이블스페이스를 여러개 생성해두고, 사용자마다 별도의 임시 테이블스페이스를 지정해 줄 수 있음

### 다. 로그파일

- DB 버퍼 캐시에 가해지는 모든 변경사항을 기록하는 파일
- 로그 기록은 Append 방식으로 이루어지기 때문에 상대적으로 매우 빠름
- 빠른 커밋 지원

### 로그파일
#### 리두로그(오라클)
- 트랜잭션의 데이터 유실 방지
- 마지막 체크포인트이후 사고 발생 직전까지 수행되었던 트랜잭션을 Redo 로그를 이용해서 재현함 (캐시복구)
- 최소 두개 이상의 파일로 구성하며 round-robin 방식 이용하여 사용

#### 트랜잭션로그(SQL Server)
- 데이터파일(데이터베이스)마다 트랜잭션 로그 파일이 하나씩 생성됨(ldf)
- 가상로그파일이라고 불리는 더 작은 세그먼트 단위로 나뉨
- 가상로그파일 개수가 너무 많아지지 않도록 옵션을 지정(로그파일을 넉넉한 크기로 만들어 자동 증가가 발생하지   
않도록 하거나, 증가단위를 크게 지정)


### Archved(=Offline) Redo 로그
#### Archived Redo 로그
- 오라클에서 온라인 리두로그가 재사용 되기 전에 다른 위치로 백업해둔 파일
- 디스크가 꺠지는 등의 물리적인 저장매체 장애에 대해서 복구하기 위해 사용
- SQL Server는 Archived Redo 로그에 대응되는 개념 없음

## 4. 메모리구조

### 시스템 공유 메모리영역
### System Global Area(SGA) == Memory Pool
- 여러 프로세스가 동시에 액세스할 수 있는 메모리 영역
- 모든 DBMS는 공통적으로 사용하는 캐시 영역이 있음(DB 버퍼캐시, 공유풀, 로그 버퍼)
- 그 외에 Large Poolm, Java Pool, 시스템 구조와 제어 구조를 캐싱하는 영역을 포함하고 있음
- 여러 프로세스가 공유되기 때문에 내부적으로 Latch, 버퍼Lock, 라이브러리 캐시 Lock/Pin같은 액세스 직렬화 매커니즘 사용

### 프로세스 전용 메모리영역
- 오라클은 프로세스 기반의 아키텍처로 서버 프로세스가 자신만의 전용 메모리 영역을 가짐 (Process Global Area(PGA))
- 데이터를 정렬하고 세션과 커서 정보를 저장
- 쓰레드기반의 아키텍처를 사용하는 SQL Server는 프로세스 전용 메모리 영역을 갖지 않는다. 

### 가. DB 버퍼캐시
- 데이터파일로부터 읽어들인 데이터 블록을 담는 캐시영역
- 사용자 프로세스는 서버 프로세스를 통해 DB 버퍼 캐시의 버퍼 블록을 동시에 액세스 (내부적으로 Buffer Lock을 통한 직렬화)
- Direct Path Read 매커니즘이 작동하는 경우를 제외하면, 모든 블록 읽기는 버퍼 캐시를 통해 이루어짐
- 디스크에서 읽을때도 버퍼캐시에 적재한 후 읽음
- 변경된 블록(더티버퍼)은 주기적으로 DBWR 프로세스에 의해 데이터파일에 기록
- 디스크 I/O는 물리적으로 액세스암이 움직이면서 헤드를 통해 이루어지는 반면, 메모리 I/O는 전기적신호에 불과하기 때문에   
  디스크 I/O와 비교할수 없을 정도로 빠름.

#### 1) 버퍼블록상태
**Free Buffer**
- 인스턴스 기둥호 아직 데이터가 읽혀지지 않아 비어 있는 상태이거나, 데이터파일과 서로 동기화 되어 언제든지 덮어써도 되는 상태

**Dirty Buffer**
- 버퍼가 캐시된 이후 변경이 발생하지만, 아직 디스크에 기록되지 않아 데이터파일 블록과 동기화가 필요한 버퍼 블록.
- 이 버퍼 블록이 재사용 되려면 디스크에 먼저 기록되어야 하고 디스크에 기록된 순간 Free 버퍼로 변경

**Pinned Buffer**
- 읽기 또는 쓰기 작업이 현재 진행중인 버퍼 블록

#### 2) LRU 알고리즘
- 버퍼 캐시는 유한한 자원이므로 모든 데이터를 캐싱해 둘 수 없기 때문에 사용 빈도가 높은 데이터 블록 위주로  
  버퍼 캐시가 구성 되도록 LRU 알고리즘을 사용
- 모든 버퍼 블록헤더를 LRU 체인에 연결해 사용 빈도 순으로 위치를 옮기다가(Touch count가 높을수록 MRU)  
  Free 버퍼가 필요해지면, 엑세스 빈도가 낮은(LRU) 쪽 데이터 블록부터 밀어내는 방식

### 나. 공유풀(shared pool)
- 딕셔너리캐시와 라이브러리 캐시로 구성되며 버퍼 캐시처럼 LRU 알고리즘을 사용

#### 1) 딕셔너리 캐시
- 테이블, 인덱스같은 오브젝트는 물론 테이블스페이스, 데이터파일, 세그먼트, 익스텐트, 사용자, 제약사항과 같은 메타정보 저장

#### 2) 라이브러리캐시
- SQL 실행에 관련된 모든 객체에 대한 정보 관리
- 서버 프로세스가 SQL을 작업할때 사용되는 작업공간
- SQL에 대한 분석정보 및 실행계획 저장
- 공유 SQL을 저장하기 위해 사용

- 라이브러리 캐시는 캐싱된 SQL과 그 실행계획의 재사용성을 높이는 것이 수행 성능을 높이고 DBMS 부하를 최소화 하는 핵심원리임
- 바인드변수 사용 및 기준에 맞는 SQL 작성으로 재사용성을 높여 줘야 함.

### 다. 로그버퍼
- Only Recovery를 위해 사용됨.
- DB버퍼에 가해지는 모든 변경사항을 로그버퍼에 먼저 기록
- **Physiolosical logging**
    - physical logging과 logical logging의 장점을 결합한것으로 변경된 데이터에 대한 before/after 이미지를 저장하고  
      opcode(명세서)를 기록하여 완벽한 복구를 보장
- **page fix rule**
    - 변경이 시작되는 시점부터 완료되는 시점까지 해당 블록을 보호해주는 아키텍처로 os에서 세마포어를 할당받아서  
      세마포어가 해당 블록을 보호
- **log a head**
    - 데이터 변경작업시에 DBWR에 의한 블록 변경보다 로그를 먼저 기록하는 기법
- **log force at commit**
    - 커밋시 리두로그를 먼저 기록하는 기법, 기록하는 속도가 빠른 리두를 먼저 기록하게 하여 중간에 발생하는   
      장애로부터 완벽한 복구를 보장하는 기법
- **logical odering of redo**
    - 로그를 기록할때 정해진 위치가 아닌 순서와 무관하게 기록하되, scn과 RBA를 이용하여 복구에 대한 순서를 결정하여 빠른 복구 보장

### 라. PGA(Process Global Area)
- 오라클의 서버 프로세스는 자신만의 PGA 메모리 영역을 할당받아 이를 프로세스에 종속적인 고유 데이터를 저장하는 용도로 사용
- PGA는 다른 프로세스와 공유되지 않은 독립적인 메모리 공간으로 똑같은 개수의 블록을 읽더라도 SGA 버퍼 캐시에서 읽는것보다 훨씬 빠름

#### 1) UGA(User Global Area)
- 각 세션을 위한 독립적인 공간
- Dedicated Server : PGA에 UGA영역 할당
- Shared Server: SGA의 Large Pool 또는 Shared Pool에 UGA 영역 할당

#### 2) CGA(Call Global Area)
- 오라클은 하나의 데이터베이스 Call을 넘어서 다음 Call까지 계속 참조되는 정보를 UGA에 담고, call이   
  진행되는 동안 필요한 데이터는 CGA에 담는다
- Parse Call, Execute Call, Fetch Call 마다 매번 할당 받음
- Call이 진행되는동안 Recursive call이 발생하면 그 안에서도 Parse, Execute, Fetch 단계별로 CGA 할당
- 할당된 공간은 Call이 끝나자마자 해제되어 PGA에 반환

#### 3) Sort Area
- 데이터 정렬을 위해 사용되며, 부족할때 마다 chunk 단위로 조금씩 할당됨
- 세션마다 sort_area_size 파라미터로 설정가능
- 9i 이상부터는 workarea_size_policy 파라미터를 auto로 설정하면 내부적으로 알아서 sort area를 할당해줌

**DML : CGA 영역에 할당**  
**SELECT : 수행중간단계에 필요한 sort area는 CGA에 할당, 최종 결과집합을 출력하기 직전 단계에서 필요한 sort area는 UGA에 할당**

## 5. 대기 이벤트
- DBMS 내부에서 활동하는 수많은 프로세스간에서는 상호작용이 필요하며, 그 과정에서 다른 프로세스가 일을 마칠때까지 기다려야하는 상황이 발생
- 그때마다 해당 프로세스는 자신이 일을 계속 진행할 수 있는 조건이 충족될때까지 수면(Sleep)상태로 대기

- **Reponse Time** = 
    - Service Time + Wait Time     
    - CPU Time + Queue Time 
    
- 서비스시간(Service Time = CPU Time): 프로세스가 정상적으로 동작하며 일을 수행한 시간
- 대기시간(Wait Time = Queue Time): 프로세스가 잠시 수행을 멈추고 대기한 시간
- Response Time Analysis 방법론은 Cpu Time과 Wait Time을 각각 Break down 하면서 서버의 일량과 대기시간을 분석
- Wait TIme은 각각 발생한 대기 이벤트를 분석해서 가장 시간을 많이 소비한 이벤트 중심으로 해결방안 모색

### 가. 라이브러리캐시 부하
- 라이브러리 캐시에서 SQL 커서를 찾고 최적화 하는 과정에서 경합이 발생하여 나타난 대기이벤트
    - latch : shared pool
    - latch : library cache
- 라이브러리 캐시와 관련해서 자주발생하는 대기 이벤트로, 수행중인 SQL이 참조하는 오브젝트에 다른 사용자가 DDL문장을 수행할 때 
    - library cache lock
    - library cache pin

### 나. 데이터베이스 call과 네트워크 부하
- 애플리케이션과 네트워크 구간에서 소모된 시간에 의해 나타난 이벤트
    - SQL*Net message from client : client로부터 다음 명령이 올떄까지 idle 상태로 기다릴때 발생   
      (데이터베이스 경합과 관계없음)
    - SQL*Net message to client : 메시지를 보냈는데 메시지를 받았다는 신호가 늦게 도착하는경우 이거나,
      클라이언트가 너무 바쁠경우
    - SQL*Net more data to client: 메시지를 보냈는데 메시지를 받았다는 신호가 늦게 도착하는 경우 이거나,
      클라이언트가 너무 바쁠경우
    - SQL*Net more data from client : 클라이언트로부터 더 받을 데이터가 있는데 지연이 발생한 경우

### 다. 디스크 부하
- 디스크 I/O 발생할 때 나타나는 대기 이벤트
    - db file sequential read : Single Block I/O call에 하나의 데이터 블록만 읽음.    
      인덱스 블록을 읽을때 발생
    - db file scattered read : Multi Block I/O. Table Full Scan 또는 Index Fast Full Scan시 나타남
    - direct path read
    - direct path write
    - direct path write temp
    - direct path read temp
    - db file parallel read

### 라. 버퍼캐시 경합
- 버퍼캐시에서 블록을 읽는 과정에서 경합이 발생하여 나타나는 대기 이벤트
    - latch : cache buffers chains
    - latch : cache buffers lru chain
    - buffers busy waits
    - free buffer waits
- 해소 방법은 I/O 부하 해소 방법과 비슷함

### 마. LOCK관련 대기이벤트 
- Lock과 관련된 대기이벤트
    - enq : TM - contention
    - enq : TX - row lock contention
    - enq : TX - index contention
    - enq : TX - allocate ITL entry
    - enq : TX contention
    - latch free : 특정 자원에 대한 래치를 여러차례(2000번 가량) 요구했지만 해당 자원이 계속 사용중이어서  
      잠시 대기 상태로 빠질때마다 발생
- Lock은 사용자 데이터를 보호하는 반면, Latch는 SGA에 공유되어 있는 갖가지 자료구조를 보호할 목적으로 사용하는 가변운 LOCK
- Latch도 일종의 Lock이지만 큐잉(Queueing) 매커니즘을 사용하지 않음
- 특정자원에 액세스하려는 프로세스는 래치 획득에 성공할때까지 반복해서 시도하나, 우선권은 부여받지 못함  
  (처음시도한 래치가 맨 나중에 래치획득에 성공할수도 있음)
- 그외 대기이벤트
    - log file sync
    - checkpoint completed
    - log file switch completion
    - log buffer space

# 제2절 SQL 파싱 부하

## 1. SQL 처리 과정

- 사용자는 구조화된 질의언어(SQL)를 통해 사용자가 원하는 결과집합을 정의
- DBMS는 사용자의 SQL을 SQL옵티마이저를 통해 실행계획을 작성해줌

### 가. SQL 파싱(Parsing)

- SQL을 실행하면 제일먼저 SQL 파서(parser)가 SQL 문장에 문법적 오류가 없는지 검사(Syntax 검사)
- 문법적 오류가 없다면 의미상 오류가 없는지 검사(Semantic 검사, 오브젝트 존재유무등)
- 검사를 다 마치면, 사용자가 발생한 SQL과 그 실행계획이 라이브러리캐시(프로시저캐시)에 캐싱되어 있는지 확인
- 캐싱되어 있다면 소프트파싱, 캐싱되어있지 않다면 하드파싱

**소프트파싱(Soft Parsing)** 
    - SQL과 실행계획을 캐시에서 찾아 곧바로 실행단계로 넘어가는 경우

**하드파싱(Hard Parsing)**
    - SQL과 실행계획을 캐시에서 찾지 못해 최적화 과정을 거치고 나서 실행단계로 넘어가는 경우
    
- 라이브러리캐시는 해시 구조로 관리됨
    - SQL마다 해시값에 따라 여러 해시 버킷으로 나뉘여 저장되고, SQL을 찾을때는 SQL 문장을 해시 함수에 적용하여  
      반환되는 해시값을 이용하여 해시 버킷을 탐색함

### 나. 최적화(Optimization)
- SQL 최적화를 담당하는 옵티마이저는 사용자가 요청한 SQL을 가장 빠르고 효율적으로 수행할 최적의(처리비용)  
  처리경로를 선택해 주는 DBMS의 핵심
  
**최적화 과정**  
- 예를들어 5개의 테이블을 조인한다면, 순서만 고려해도 5!(=120)개의 실행계획 평가
- 120가지의 실행계획에 포함된 각 단계별 다양한 조인방식 고려
- 테이블을 full scan 할지 인덱스를 사용할지, 어떤 인덱스를 어떤방식으로 스캔할지 고려
    - 이와 갘이 무거운 작업이므로 이러한 힘든 과정을 거쳐 최적화된 SQL 실행계획을 한번만 쓰고 버린다면  
      엄청난 비효율이 발생한다.
    - 파싱과정을 거친 SQL과 실행계획이 여러 사용자가 공유해서 재사용할 수 있도록 공유메모리에 캐싱은 이유가 여기에 있다.

## 2. 캐싱된 SQL 공유

### 가. 실행계획 공유 조건
- SQL 수행절차
    - 문법적 오류와 의미상 오류가 없는지 검사
    - 해시 함수로붜 반환받은 해시 값으로 라이브러리 캐시 내 해시버킷 탐색
    - 찾아간 해시버킷에 체인으로 연결된 엔트리를 차례로 스캔하면서 같은 SQL 문장 탐색
    - SQL 문장을 찾으면 함께 저장된 실행계획을 가지고 바로 실행
    - 찾아간 해시 버킷에서 SQL 문장을 찾지 못하면 최적화를 수행
    - 최적화를 거친 SQL과 실행계획을 방금 탐색한 해시 버킷 체인에 연결
    - 방금 최적화한 실행계획을 가지고 실행

**중요**
- 하드파싱을 반복하지 않고 캐싱된 버전을 찾아 재사용하려면 SQL을 먼저 찾아가야 하며, 캐시에서 SQL을 찾기위해 사용되는 키값은 SQL 문장 그자체
  => 이 때문에 SQL 문장안의 작은 공백 하나로도 DBMS는 서로 다른 SQL 문장으로 인식할수 있으므로 주의 해야함

### 나. 실행계획이 공유하지 못하는 경우
- 1. 공백 또는 줄바꿈
- 2. 대문자 구분
- 3. 주석(Comment)
- 4. 테이블 Owner 명시
- 5. 옵티마이저 힌트사용
- 6. 조건절 비교값

- 이러한 비효율을 줄이고 공유 가능한 형태로 SQL을 작성하려면 개발 초기에 SQL 작성표준을 정해서 이를 준수하도록 해야함
- 6번처럼 조건절값을 문자열로 붙여가며 매번 다른 SQL로 실행되는 리터널 SQL의 경우, 한가한 시간이라면 문제에 대해서  
느끼지 못하겠지만, 사용자가 동시에 몰리는 시간대에는 장애상황으로 발생할 수도 있으므로 바인드변수의 사용을 고려해야 함

## 3. 바인드 변수 사용하기

### 가. 바인드 변수의 중요성 

**바인드 변수를 사용했을떄의 효과**
- SQL과 실행계획을 반복적으로 재사용함으로써 파싱 소요시간과 메모리 사용량을 줄여줌
- 궁극적으로 시스템전반의 CPU와 메모리 사용률을 낮춰 데이터베이스 성능과 확장성을 높임

**바인드 변수를 사용하지 않아도 되는 예외상황**
- 배치프로그램이나 DW, OLAP등 정보계 시스템에서 사용되는 Long Runnung 쿼리
    - 파싱 소요시간이 총 소요시간에서 차지하는 비중이 낮음
    - 수행빈도가 낮아 하드파싱에 의한 라이브러리 캐시 부하 유발 가능성이 낮음
    - 그러므로 상수조건절을 사용하여 옵티마이저가 컬럼히스토그램 정보를 활용할 수 있도록 유도하는것이 유리함
- 조건절 컬럼의 값 종류(Distinct value)가 소수 일때
    - 분포도가 좋지 않은 값은 옵티마이저가 컬럼히스토그램 정보를 활용할 수 있도록 유도.
- 이러한 경우가 아니라면 OLTP 환경에서는 바인드 변수 사용을 권고함
- 리터널 SQL을 자동으로 변수화 시켜주는 기능

### 나. 바인드변수 사용시 주의사항
- 칼럼의 분포가 균일할때는 바인드 변수 처리가 나쁘지 않음
- 칼럼의 분포가 균일하지 않을때에는 실행 시점에 바인딩되는 값에 따라 쿼리 성능이 다르게 나타날 수 있으므로  
  이럴때는 상수값을 사용하는것이 나을 수 있음

### 다. 바인드 변수 부작용을 극복하기 위한 노력
- 바인드변수 Peeking 기능 도입: 첫번째 바인드 변수값을 살짝 훔쳐보고 그 값에 대한 분포를 이용하여 실행계획 결정하는 기능
- 이또한 처음 훔쳐본값에 따라 비활성화 시켜 사용하고 있음
- 오라클은 11g부터는 적응적 커서공유(Adaptive Cusor Sharing)를 도입하여 칼럼 분포에 따라 다른 실행계획이  
  사용되도록 처리하였지만 이또한 완전한 기능이 아니므로 주의해서 사용해야 한다.

## 4. Static SQL과 Dynamic SQL

### 가. Static SQL
- String형 변수에 담지 않고 코드 사이에 직절 기술한 SQL문 (Embedded SQL)
- 개발언어 : PowerBuilder, PL/SQL, Pro*C, SQLJ
- SQL문을 String 변수에 담지 않고 마치 예약된 키워드처럼 C/C++코드 사이에 섞어 기술
- 구문분석, 유효 오브젝트 여부, 오브젝트 엑세스 권한등의 체크 가능

### 나. Dynamic SQL
- String 형 변수에 담아서 기술하는 sQL문
- 조건에 따라 SQL이 동적으로 바뀔수 있으므로 syntax, semantics 체크 불가능

### 다. 바인드 변수의 중요성 재강조
- Static을 사용하든 Dynamic SQL을 사용하든 옵티마이저는 SQL 문장 자체만을 인식할 뿐이므로 성능에 영향을 주지는 않는다.
- 라이브러리 캐시 효율은 Static이냐 Dynamic의 차이가 아니라 바인드 변수의 사용여부에 초점을 맞춰야 함

## 5. 애플레키에션 커서 캐싱
- 같은 SQL을 여러번 반복해서 수행해야할 때, 첫번째는 하드파싱이 일어나겠지만 이후부터는 라이브러리 캐시에 공유되 버전을 찾아 가볍게 실행한다.
- 하지만 그렇다더라도 SQL문장의 문법적, 의미적 오류를 확인하고 해시함수로부터 반환된 해시값을 이용해서 캐시에서 실행계획을 찾고, 수행이 필요한 메모리를 할당받는 등의 작업이 매번 반복되면 비효율이 발생할 것이다.
- 이러한 과정을 생략하고 빠르게 SQL을 수행하는 방법이 바로 "**애플리케이션 커서 캐싱**" 이다.
- 개발언어마다 구현방식이 다르므로 이 기능을 활용하려면 API를 살펴봐야함.
- 일반적으로 SQL을 반복 수행할 때에는 Parse Call 횟수가 Execute Call 횟수와 같지만 
- 위의 결과는 Parse Call 한번만 발생했고, 이후 4,999번 수행할 때에도 Parse Call이 전혀 발생하지 않았음
- JAVA에서 위의 기능을 구현하기 위한 방법 : 묵시적캐싱 옵션 사용(Implicit Caching)
- Dynamic SQL을 사용하거나 Cursor Variable(=Ref Cursor)를 사용할 때는 커서를 자동으로 캐싱하는 효과가 사라짐

# 제 3절. 데이터베이스 Call과 네트워크 부하

# 1. 데이터베이스 Call 종류

## 가. SQL 커서에 대한 작업 요청에 따른 구분
- Parse Call : SQL 파싱을 요청하는 Call
- Execute Call : SQL 실행을 요청하는 Call
- Fetch Call : SELECT 문의 결과 데이터 전송을 요청하는 Call

## 나. Call 발생 위치에 따른 구분
### 1) User Call
- DBMS로부터 요청되는 Call
- User Call이 많으면 성능이 좋을수 없으므로, DBMS 확장성을 높이려면 User Call을 최소화 하려는 노력이 중요함
- User Call을 줄이기 위한 기술요소
    - Loop 쿼리를 해소하고 집합적 사고를 통해 One SQL로 구현
    - Array Processing : Array 단위 Fetch, Bulk Insert/Update/Delete
    - 부분범위처리 원리 활용
    - 효과적인 화면 페이지 처리
    - 사용자 정의 함수/프로시저/트리거의 적절한 활용

### 2) Recusive Call
- DBMS 내부에서 발생하는 Call
- SQL 파싱과 최적화 과정에서 발생(데이터 딕셔너리조회, 사용자 정의함수/프로시저 내에서의 SQL 수행)
- Recursive Call 최소화 방안
    - 바인드 변수 사용하여 하드파싱 발생 횟수 감소
    - 사용자 정의 함수/프로시저의 적절한 사용 

# 2. 데이터베이스 Call과 성능

## 가. One SQL 구현의 중요성

- 예를 들어 처리해야할 납부실적이 10만건이라고 하면, (array 단위 fetch 기능을 이용하지 않을때 Insert를 위한 Parse Call이 50만번, Execute call이 50만번으로 최대 110만 번의 데이터베이스 Call이 발생함
- 이러한 프로그램을 아래와 같이 One SQL로 통합하면 1~2초내로 수행되는것을 확인할수 있는데, 이 원리는 110번 발생할수 있는 데이터베이스 Call을 단 2회(Parse Call 1회, Execute Call 1회)로 줄인데 있다

## 나. 데이터베이스 Call과 시스템 확장성
- 데이터베이스 Call은 개별 프로그램 수행속도 뿐 아니라 궁극적으로 시스템 전체의 확장성에도 영향을 미친다.
- 예시) 인터넷 쇼핑몰에서 조회한 상품 5개를 선택한 후 위시리스트에 등록하는 프로그램일 때
- 5번의 메소드를 호출하므로 Parse Call과 Execute Call 각각 5번씩 발생
- 확장성을 고려하여 작성하였으므로 1번의 메소드를 호출하며 Parse Call과 Execute Call도 각각 1번씩만 호출

# 3. Array Processing 활용
- Array Processing 기능을 활용하면 한번의 SQL(INSERT/UPDATE/DELETE) 수행으로 다량의 레코드를 동시 처리
- 네트워크 Call 감소
- SQL 수행시간 감소
- CPU 사용량 감소

- Insert 할 데이터를 계속 array에 담기만 하다가
- 1000건이 쌓일때마다 한번씩 executeBatch를 수행
- Select 결과집합을 Fetch할때도 1000건씩 하도록 조정하였다.  
  **Call 횟수를 줄이는것이 성능개선에 도움이 되는것을 알 수 있다.**
- 대용량 데이터의 처리에는 Array Processing이 필수
- 효과를 극대화 하기위해 연속된 일련의 처리과정을 모두 Array 단위로 진행해야함(select, insert 모두)
- 예시) PL/SQL을 이용한 데이터 Bulk로 1000건씩 Fetch해서 Bulk Insert

# 4. Fetch Call 최소화

## 가. 부분범위처리 원리
- 쿼리 결과 집합을 전송할때, 전체 데이터를 연속적으로 전송하지 않고 사용자로부터 Fetch Call이 있을때마다 일정량씩 나누어서 전송하는 것
- 데이터를 클라이언트에게 전송할때 일정량씩 나누어서 전송
- 오라클 : 내부적으로는 SDU, TDU 단위로 나누어서 전송  
           array 사이즈를 작게 설정하면 하나의 네트워크 패킷에 담아 전송하겠지만, 크게 설정하면 여러개의 패킷으로 나누어 전송

## 나. ArraySize 조정에 의한 Fetch Call 감소 및 블록 I/O 감소 효과
- 대량의 데이터파일을 전송한다면 arraysize 크게하여 fetch call 횟수를 줄여주는것이 유리
- 반대로 적은량의 데이터만 fetch 하다가 멈추는 프로그램이라면 arraysize를 작게 설정하는것이 유리
- arraysize를 증가시키면 네트워크 부하감소 및 서브프로세스가 읽어야할 블록 갯수 감소 효과
- ArraySize를 키운다고 해서 Fetcah count와 블록 I/O가 같은 비율로 줄지 않음
- ArraySize를 무작정 크게 설정한다고 좋은것이 아니며, 일정크기 이상이면 리소스만 낭비하는 결과를 초래할 수 있음

### 1) Oracle PL/SQL(커서를 열고 레코드를 Fetch)
- 10g : 자동으로 100개씩 array Fetch가 일어남.

### 2) JAVA(FetchSize를 100으로 설정했을때 데이터를 Fetch 해오는 매커니즘)
- 최초 rs.next() 호출 시 한꺼번에 100건을 가져와서 클라이언트 Array 버퍼에 캐싱한다.
- 이후 rs.next() 호출할 때는 데이터베이스 Call을 발생시키지 않고 Array 버퍼에서 읽는다.
- 버퍼에 캐싱 돼 있던 데이터를 모두 소진한 후 101번째 rs.next() 호출 시 다시 100건을 가져온다.
- 모든 결과집합을 다 읽을 때까지 2~3번 과정을 반복한다.

# 5. 페이지 처리 활용
- 페이지 처리를 하지 않을때 발생하는 부하요인
    - 다량 발생하는 Fetch Call의 부하
    - 대량의 결과 집합을 클라이언트에 전송하면서 발생하는 네트워크 부하
    - 인덱스와 부분범위처리 원리를 이용해 각 페이지에 필요한 최소량만 I/O
- 데이터를 소량씩 나누어 전송하므로 AP웹 서버 리소스 사용량 최소화
- 결론적으로, 조회할 데이터가 일정량 이상이고 수행빈도가 높다면 필수적으로 페이지 처리 구현해야함

# 6. 분산 쿼리
- 분산 DB간 테이블 조인
- 원격의 sal테이블을 전송받아 order 테이블과 NL 조인
- 50만건이나 되는 데이터를 네트워크를 통해 전송받고 있어 성능저하의 원인이 됨.
- 분산 DB간의 성능저하 해결방안
- order_data에 조건에 해당하는 데이터만 원격으로 보내서 조인과 group by를 거친 결과 집합만 전송받음
- 원격서버가 처리가능 하도록 driving_site 힌트 사용
- 분산쿼리의 성능을 높이는 핵심은, 네트워크를 통한 데이터 전송량을 줄이는 데 있다.

# 7. 사용자 정의 함수 / 프로시저의 특징과 성능

# 가. 사용자 정의 함수/프로시저의 특징
- 가상머신과 같은 별도의 실행엔진을 통해 실행됨
- 실행시마다 컨텍스트 스위칭이 일어나므로, 내장함수를 호출할때와 비교해서 성능이 상당히 떨어짐
- 예시) 문자타입의 일자 데이터를 날짜 타입으로 변환해주는 사용자정의함수
- to_char 함수를 바로 호출할때보다 훨씬 느림
- 메인쿼리가 참조하는 사용자 정의 함수에 또 다른 쿼리문이 내장되어 있다면 수행 성능이 훨씬 나빠짐
- 함수에 내장된 쿼리를 수행할 때마다 Execute Call, Fetch Call이 재귀적으로 일어남
- Recusive Call이 반복적으로 일어남(User Call에 비해 성능부하가 미미하지만, 그 횟수가 무수히 반복되면 성능저하)

## 나. 사용자 정의 함수/프로시저에 의한 성능 저하 해소 방안
- 소량의 데이터를 조회할 때 사용
- 부분범위 처리가 가능한 상황에서 제한적으로 사용
- 가급적 함수를 풀어 조인 또는 스칼라 서브쿼리 형태로 변환

# 제4절. 데이터베이스 I/O원리

## 1. 블록단위 I/O
- 데이터 파일에서 DB 버퍼 캐시로 블록을 적재할 때
- 데이터 파일에서 블록을 직접 읽고 쓸때
- 버퍼 캐시에서 블록을 읽고 쓸때
- 버퍼 캐시에서 변경된 블록을 다시 데이터 파일에 쓸때

## 2. 메모리 I/O vs. 디스크 I/O

### 가. I/O 효율화 튜닝의 중요성
- 디스크를 경우한 데이터 입출력은 디스크의 액세스 암(Arm)이 움직이면서 헤드를 통해 데이터를 읽고 쓰기 때문에 느림
- 메모리를 통한 입출력은 전기적 신호에 불과하기 때문에 디스크를 통한 I/O에 대해 비교할수 없을정도로 빠름
- 모든 DBMS는 읽고자 하는 블록을 먼저 버퍼 캐시에서 찾고, 없을경우에는 디스크에서 읽어 버퍼 캐시로 적재후 읽기/쓰기 작업을 수행
- 이러한 이유로 모든 데이터를 메모리에 올려놓고 사용하면 좋겠지만 메모리는 물리적으로 한정된 자원
- **결국 디스크 I/O를 최소화하고 버퍼 캐시 효율을 높이는 것이 데이터베이스 I/O 튜닝의 목표**

### 나. 버퍼 캐시 히트율(Buffer Cache Hit Ratio)
- 버퍼 캐시 효율을 측정하는 지표로써 전체 읽은 블록중 메모리 버퍼 캐시에서 찾은 비율   
  BCHR = (버퍼 캐시에서 곧바로 찾은 블록 수 / 총 읽은 블록 수) x 100
- BCHR은 주로 전체적인 관점에서 측정하지만, 개별 SQL에 대해서도 구해볼수 있으며, 이 비율이 낮은것이 SQL 성능을 떨어뜨리는 주범
- Disk 항목이 디스크를 경유한 블록 수
    - 총 읽은 블록 수 = 822
    - 버퍼 캐시에서 곧바로 찾은 블록 수 = 822 - 18 = 804
    - CHR = (822 - 18) / 822 = 97.8%
- **논리적인 블록요청 횟수를 줄이고, 물리적으로 디스크에서 읽어야할 블록수를 줄이는것이 I/O 효율화 튜닝의 핵심 원리**

### 다. 네트워크, 파일시스템 캐시가 I/O 효율에 미치는 영향
- 대용량의 데이터를 읽고 쓰는데 다양한 네트워크 기술이 사용됨에 따라 네트워크 속도도 SQL 성능에 크게 영향을 미친다.
- RAC같은 클러스터링 환경에선 인스턴스 간에 캐시된 블록을 공유하므로 메모리 I/O의 성능에도 네트워크 속도가 지대한 영향을 미친다. 
- 같은양의 디스크 I/O가 발생하더라도 I/O 대기시간이 크게 차이 나는것은 디스크 경합 때문일수도 있지만, 
  OS에서 지원하는 파일시스템 버퍼 캐시와 SAN 캐시 때문일 수도 있다.
- (SAN 캐시는 크다고 문제가 되지 않지만, 파일시스템 버퍼 캐시는 최소화 하여 데이터베이스 자체 캐시영역에 좀더 큰   
  공간을 할당하는것이 더욱 효과적임)
- **네트워크 문제든, 파일시스템 문제든 I/O 성능에 관한 가장 확실하고 근본적인 해결책은 논리적인블록 요청 횟수를 최소화 하는것**

## 3. Sequential I/O vs. Random I/O
- sequential 액세스는 논리적/물리적 순서를 따라 차례대로 읽어 나가는 방식
    - 인덱스 리프 블록에 위치한 모든 레코드는 포인터를 논리적으로 연결되어 있고, 이 포인터를 따라 스캔하는 방식
- Random 액세스는 레코드간 논리적, 물리적인 순서를 따르지 않고, 한건을 읽기 위해 한 블록씩 접근하는 방식
- I/O 튜닝의 핵심 원리
    - Sequential 액세스에 의한 선택 비중을 높인다.
    - Random 액세스 발생량을 줄인다.

### 가. Sequential 액세스에 의한 선택 비중 높이기
- 읽는 총 건수 중에서 결과 집합으로 선택되는 비중을 높여야 함
- 같은 결과를 얻기위해 얼마나 적은 레코드를 읽느냐로 효율성이 판단됨

- good
    - 전체 레코드 49,906건
    - 선택 레코드 24,613건 (49%)
    - 읽은 블록수 691 블록
    - Table Full Scan 치고는 나쁘지 않음
- bad    
    - 전체 레코드 49,906건
    - 선택 레코드 1건 (0.002%)
    - 읽은 블록수 691 블록
    - Table Full Scan 비효율 발생

- 인덱스를 사용하고도 1개의 레코드를 읽기위해 76개 블록 액세스
- 테이블 뿐 아니라, 인덱스를 sequential 액세스 방식으로 스캔할 때도 비효율 발생
- 조건절에 사용된 컬럼과 연산자 형태, 인덱스 구성에 의해 효율성이 결정됨

### 나. Random 액세스 발생량 줄이기
- Random 액세스 발생량을 낮추는 방법
    - 인덱스로부터 만족하는 22,934건을 읽어 그 횟수만큼 테이블을 Random 액세스 수행하여 최종적으로 1건의 결과 추출
    - 최종 선택된 것에 비해 너무 많은 Random 액세스 발생

## 4. Single Block I/O vs.MultiBlock I/O
- single Block I/O : 한번의 I/O Call에 하나의 데이터 블록만 읽어서 메모리에 적재하는 방식
    - 인덱스를 통해 테이블을 액세스할때, 기본적으로 인덱스와 테이블 모두 이 방식 사용
- Multi Block I/O: I/O Call이 필요한 시점에, 인접한 블록들을 같이 읽어 메모리에 적재하는 방식
    - Table Full Scan 처럼 물리적으로 저장된 순서에 따라 읽을때 인접한 블록들을 같이 읽는것이 유리함
    - 인접한 블록 : 하나의 익스텐트에 속한 블록

**Single Block I/O 방식**
- 64번의 인덱스 블록을 디스크에서 읽으면서 64번의 I/O Call이 발생

**MultiBlock I/O 방식**
- 64개 블록을 디스크에서 읽었는데 I/O Call이 9번 발생
- Oracle 10g부터는 Index Range Scan 또는 Index Full Scan일때도 Multiblock I/O 방식으로 읽는 경우가 있음

## 5. I/O 효율화 원리
- **논리적인 I/O 요청 횟수를 최소화 하는것이 I/O 효율화 튜닝의 핵심 원리**
- I/O 때문에 성능이 낮게 측정될때, 하드웨어적인 방법을 통해 I/O 성능을 향상 시킬수도 있지만,  
  SQL 튜닝을 통해 I/O 발생 횟수를 줄이는것이 근본적이고 확실한 해결 방법이다. 
- 애플리케이션 측면에서 이 I/O 효율화 원리
    - 필요한 최소 블록만 읽도록 SQL 작성
    - 최적의 옵티마이징 팩터 제공 
    - 필요하다면, 옵티마이저 힌트를 사용하여 최적의 액세스 경로를 유도

### 가. 필요한 최소 블록만 읽도록 SQL 작성
- 비효율적인 액세스를 없애고 필요한 최소 블록만 액세스
- 같은 테이블을 4번 액세스하여 처리하던 방식을 1번만 읽어서 처리할수 있도록 튜닝

### 나. 최적의 옵티마이징 팩터 제공
- 전략적인 인덱스 구성
    - 전략적인 인덱스 구성은 가장 기본적인 옵티마이징 팩터
- DBMS가 제공하는 기능 활용
    - 인덱스 외에도 DBMS가 제공하는 다양한 기능을 적극적으로 활용
    - 인덱스, 파티션, 클러스터, 윈도우 함수 등을 적극 활용하여 옵티마이저가 최적의 선택을 할 수 있도록 한다. 
- 옵티마이저 모드 설정
    - 옵티마이저모드(전체처리속도, 최초등담속도)와 그 외 옵티마이저 행동에 영향을 미치는 일부 파라미터를 변경해 주는것이 도움이 될 수 있다.
- 통계정보
    - 옵티마이저에게 정확한 정보를 제공

### 다. 필요하다면, 옵티마이저 힌트를 사용해 최적의 액세스 경로로 유도
- 최적의 옵티마이징 팩터를 제공했다면 가급적 옵티마이저에게 맡기는것이 바람직하지만,   
  옵티마이저가 생각만큼 최적의 실행계획을 수립하지 못하는 경우 사용
- 옵티마이저 힌트를 사용할 때 반드시 의도한 실행계획으로 수행되는지 확인해야 함
- 여러가지로 옵티마이저 힌트 사용은 불가피함
- 데이터베이스 애플리케이션 개발자라면 인덱스, 조인, 옵티마이저의 기본 원리를 이해 필요
- 그것을 바탕으로 최적의 액세스 경로를 유도할 수 있는 능력을 필수적으로 갖춰야함

# 2장. Lock과 트랜잭션 동시성 제어

# 제 1절. Lock

# 1. Lock

## 가. Lock이란?
- 같은 자원을 액세스하려는 다중 트랜잭션 환경에서 데이터베이스의 일관성과 무결성을 유지하기 위해 트랜잭션의   
  순차적 진행을 보장할 수 있는 직렬화(Serialization) 장치

## 나. 공유 Lock과 배타적 Lock

### 1) 공유 Lock
- 공유(Shared) Lock은 데이터를 읽고자 할 때 사용
- 다른 공유 Lock과는 호환되지만 배타적 Lock과는 호환되지 않음

### 2) 배타적 lock
- 배타적(Exclusive) Lock은 데이터를 변경하고자 할 때 사용되며, 트랜잭션이 완료될 때까지 유지
- 해당 Lock이 해제될 때까지 다른 트랜잭션은 해당 Resource에 접근할 수 없음

## 다. 블로킹과 교착상태
### 1) 블로킹
- Lock경합이 발생해 특정 세션이 작업을 진행하지 못하고 멈춰 선 상태
- 공유 Lock과 배타적 Lock은 함께 설정될 수 없으므로 Blocking이 발생
- Blocking을 해소 할 수 있는 방법은 COmmit(또는 Rollback)뿐이다.
- Lock 경합이 발생하면 먼저 Lock이 완료될 때까지 후행 트랜잭션을 기다려야 한다. 
- Lock에 의한 성능 저하를 최소화 하는 방법
    - 트랜잭션의 원자성을 훼손하지 않는 선에서 트랜잭션을 가능한 짧게 정의
    - (Oracle은 데이터를 읽을 때 Shared Lock을 사용하지 않기 때문에 상대적으로 Lock 경합이 적음)
    - 같은 데이터를 갱신하는 트랜잭션이 동시에 수행되지 않도록 설계 
    - 주간에 대용량 갱신 작업이 불가피하다면, 블로킹 현상에 의해 사용자가 무한정 기다리지 않도록 적절한 프로그램 기법을 도입
- 트랜잭션 격리성 수준(Isolation Level)를 불필요하게 상향 조정하지 않는다. 
- SQL문장이 가장 빠른ㄴ 시간 내에 처리를 완료하도록 하는 것이 Lock 튜닝의 기본이고 효과도 가장 좋다.

### 2) 교착상태
- 두 세션이 각각 Lock을 설정한 리소스를 서로 액세스하려고 마주보며 진행하는 상황, 둘 중 하나가 뒤로 물러나지 않으면 영영 풀릴 수 없다.
- 여러 테이블을 액세스하면 발생하는 교착상태는 테이블 접근 순서를 같게 처리하여 회피 한다. 
- SQL Server라면 갱신(Update) Lock을 사용함으로써 교착상태 발생 가능성을 줄일 수 있음

# 2. SQL Server Lock

## 가. Lock 종류

### 1) 공유 Lock
- SQL Server의 공유 Lock은 트랜잭션이나 쿼리 수행이 완료될 때까지 유지되는 것이 아니라 다음 레코드가 읽히면 곧바로 해제 된다. 
- Isolation Level을 변경하지 않고 트랜잭션 내에서 공유 Lock이 유지되도록 하려면 테이블 힌트로 지정하면 된다.

### 2) 배타적 Lcok
- 데이터를 변경시 사용

### 3) 갱신 Lock
- 만약 두 트랜잭션이 동시에 같은 고객에 대해서 Update를 수행시 두 트랜잭션 모두 처음에는 공유 Lock을 설정했다가 적립포인트를 변경하기  
  직전에 배타적 Lock을 설정하려고 한다.
- 이럴 경우 두 트랜잭션은 상대편 트랜잭션에 의한 공유 Lock이 해제되기만을 가디라는 교착상태에 빠지게 된다. 
- 이런 잠재적인 교착상태를 방지하려고 SQL Server는 갱신(Update)Lock을 사용할 수 있다.
- 한 자원에 대한 갱신 Lock은 한 트랜잭션만 설정할 수 있다. 

### 4) 의도 Lock
- 특정 로우에 Lock을 설정하면 그와 동시에 상위 레벨 개체(페이지, 익스텐트, 테이블)에 내부적으로 의도(Intent)Lock이 설정된다.
- Lock을 설정하려는 개체의 하위 레벨에서 선행 트랜잭션이 어떤 작업을 수행 중인지를 알리는 용도로 사용되며, 일조의 푯말(Flag)라고 할 수 있다.
- 예를 들어, 구조를 변경하기 위해 테이블을 잠그려 할 때 그 하위의 모든 페이지나 익스텐트, 로우에 어떤 Lock이 설정돼 있는지 검사할 경우 
  오래 소요 될 수 있으므로 해당 테이블에 어떤 모드의 의도 Lock이 설정돼 있는지만 보고도 작업을 진행할지 아니면 기다릴지를 결정할 수 있다.

### 5) 스키마 Lock
- Sch-S(Schema Stability) : SQL을 컴파일하면서 오브젝트 스키마를 참조할 때 발생하며, 읽는 스키마 정보가 수정되거나 삭제되지 못하도록 함
- Sch-M(Schema Modification) : 테이블 구조를 변경하는 DDL문을 수행할 때 발생하며, 수정 중인 스키마 정보를 다른 세션이 참조하지 못하도록 함

### 6) Bulk Update Lock
- 테이블 Lock의 일종으로, 테이블에 데이터를 Bulk Copy할 때 발생한다.
- 병렬 데이터 로딩(Bulk Insert나 bcp 작업을 동시 수행)을 허용하지만 일반적인 트랜잭션 작업은 허용되지 않는다. 

## 나. Lock 레벨과 Escalation

- 로우 레벨
    - 변경하려는 로우(실제로는 RID)에만 Lock을 설정하는 것
- 페이지 레벨
    - 변경하려는 로우가 담긴 데이터 페이지(또는 인덱스 페이지)에 Lock을 설정하는 것
    - 같은 페이지에 속한 로우는 진행 중인 변경 작업과 무관하더라도 모두 잠긴것과 같은 효과가 나타남
- 익스텐트 레벨
    - 익스텐트 전체가 잠김
    - SQL Server의 경우, 하나의 익스텐트가 여덞 개 페이지로 구성되므로 8개 페이지에 속한 모든 로우가 잠긴 것과 같은 효과가 나타남
- 테이블 레벨
    - 테이블 전체 그리고 관련 인덱스까지 모두 잠김
- 데이터베이스 레벨
    - 데이터베이스 전체가 잠긴다.
    - 보통 데이터베이스를 복구하거나 스키마를 변경할 때 일어 남

- 위 5가지 레벨 외에 인덱스 키에 로우 레벨 Lock을 거는 경우도 있음
- Lock Escalation
    - 관리할 Lock 리소스가 정해진 임계치를 넘으면 로우 레벨 락이 -> 페이지 -> 익스텐트 -> 테이블레벨 락으로 점검 확장되는 것을 의미
- SQL Server, DB2 UDB 처럼 한정된 메모리 상에서 Lock 매니저를 통해 Lock 정보를 관리하는 DBMS에서 공통적으로 발생할 수 있는 현상
- Locking 레벨이 낮을 수록 동시성은 좋지만 관리해야 할 Lock 개수가 증가하기 때문에 더 많은 리소스를 소비
    - Locking 레벨이 높을 수록 적은 양의 Lock 리소스를 사용하지만 하나의 Lock으로 수많은 레코드를 한꺼번에 Locking하기 때문에 동시성은 나빠짐

## 다. Lock 호환성

- Intent Shared(IS) == IS, S, U, IX, SIX
- Shared(S) == IS, S, U
- Updated(U) == IS, S
- Intent Exclusive(IX) == IS, IX
- Shared with intent exclusive(SIX) == IS
- EXclusive(X)

- 스키마 Lock 호환성
    - Sch-S는 Sch-M을 제외한 모든 Lock과 호환
    - Sch-M은 어떤 Lock과도 호환되지 않음

# 3. Oracle Lock
- Oracle은 공유 리소스와 사용자 데이터를 보호할 목적으로 DML Lock, DDL Lock, 래치(Latch), 버퍼 Lock, 라이브러리 캐시  
  Lock/Pin등 다양한 종류의 Lock을 사용

## 가. 로우 Lock

- Oracle에서 로우 Lock은 항상 배타적이다.
- INSERT, UPDATE, DELETE 문이나 SELECT...FOR UPDATE문을 수행한 트랜잭션에 의해 설정되면, 트랜잭션이 커밋 또는 롤백할 때까지   
  다른 트랜잭션은 해당 로우를 변경할 수 없음
- Oracle에서 읽는 과정에서는 어떤 Lock도 설정하지 않음으로 읽기와 갱신 작업은 서로 방해 하지 않음
    - 읽으려는 데이터를 다른 트랜잭션이 갱신 중이더라도 기다리지 않음
    - 갱신하려는 데이터를 다른 트랜잭션이 읽는 중이더라도 기다리지 않음(SELECT..FOR UPDATE 구문은 제외)
    - 갱신하려는 데이터를 다른 트랜잭션이 갱신중이면 기다림
- Oracle이 공유 Lock을 사용하지 않고도 일관성을 유지할 수 있는 것은 UNDO 데이터를 이용한 다중 버전 동시성 제어 매커니즘을 사용하기 때문.
- Oracle은 별도의 Lock 매니저 없이 레코드의 속성으로서 로우 Lock을 구현했기 때문에 아무리 많은 레코드를 갱신하더라도 절대 Lock Escalation은 발생하지 않음

## 나. 테이블 Lock

- 한 트랜잭션이 로우 Lock을 얻는 순간, 해당 테이블에 대한 테이블 Lock도 동시에 얻어 현재 트랜잭션이 갱신중인 테이블에 대한 호환되지 않는  
  DDL 오퍼레이션을 방지 한다.
- 테이블 Lock 종류
    - Row Share(RS)
    - Row Exclusive(RX)
    - Share(S)
    - Share row Exclusive(SRX)
    - Exclusive(X)
- SELECT..FOR UPDATE 문을 수행할 때 RS 모드 테이블 Lock을 얻고, insert, update, delete 문을 수행할 때 RX 모드 테이블 Lock을 얻음
- 일반적으로 DML 로우 Lock을 처음 얻는 순간 묵시적으로 테이블 Lock을 얻지만, 아래처럼 명령어를 이용해서도 가능
- 테이블 Lock이라 하면, 테이블 전체에 Lock이 걸린다고 생각하기 쉬우나, Oracle의 테이블 Lock의 의미는, Lock을 획득한 선행 트랜잭션이 해당 테이블에서 현재 어떤 작업을 수행중인지를 알리는 일종의 푯말(Flag)이다.   
  후행 트랜잭션은 어떤 테이블이 Lock이 설정돼 있는지만 보고도 그 테이블로의 진입 여부를 결정할 수 있다. 
- Oracle의 Lock 호환성
    - NULL == NULL, RS, RX, S, SRX, X
    - RS == NULL ,RS, RX, S, SRX
    - RX == NULL, RS, RX
    - S == NULL, RS, S
    - SRX == NULL, RS
    - X == NULL

# 제 2절 트랜잭션
- 트랜잭션(Transaction)은 업무 처리를 위한 논리적인 작업 단위이다.

## 1. 트랜잭션의 특징
- 원자성(Atomicity)
    - 트랜잭션은 더 이상 분해가 불가능한 업무의 최소단위이므로, 전부 처리되거나 아예 하나도 처리되지 않아야 함
- 일관성(Consistency)
    - 일관된 상태의 데이터베이스에서 하나의 트랜잭션을 성공적으로 완료하고 나면 그 데이터베이스는 여전히 일관된 상태여야 함.
- 격리성(Isolation)
    - 실행 중인 트랜잭션의 중간 결과를 다른 트랜잭션이 접근할 수 없음
- 영속성(Durability)
    - 트랜잭션이 일단 실행을 성공적으로 완료하면 그 결과는 데이터베이스에 영속적으로 저장.

## 2. 트랜잭션 격리성

## 가. 낮은 단계의 격리성 수준에서 발생할 수 있는 현상들

### 1) Dirty Read
- 다른 트랜잭션에 의해 수정됐지만 아직 커밋되지 않음 데이터를 읽는 것을 의미
- 변경 후 아직 커밋되지 않은 값을 읽었는데 변경을 가한 트랜잭션이 최종적으로 롤백된다면 그 값을 읽은 트랜잭션은 비일관된 상태에 놓이게 됨

### 2) Non-Repetable Read
- 한 트랜잭션 내에서 같은 쿼리를 두 번 수행했는데, 그 사이에 다른 트랜잭션이 값을 수정 또는 삭제하는 바람에 두 쿼리에 결과가 다르게 나타나는 현상

### 3) Phantom Read
- 한 트랜잭션 내에서 같은 쿼리를 두 번 수행했는데, 첫 번째 쿼리에서 없던 유령(Phantom) 레코드가 두 번째 쿼리에서 나타나는 현상
- TX1 트랜잭션이 지역별고객과 연령대별 고객을 연속해서 집계하는 도중에 새로운 고객이 TX2 트랜잭션에 의해 등록
- 그 결과, 지역별고객과 연령대별 고객 두 집계 테이블을 통해 총고객수를 조회하면 서로 결과 값이 다름

## 나. 트랜잭션 격리성 수준(Transaction Isolation Level)

- **Read Uncommitted** 
    - 트랜잭션에서 처리 중인 아직 커밋되지 않음 데이터를 다른 트랜잭션이 읽는 것을 허용
    - Dirty Read, Non-Repeatable Read, Phantom Read
- **Read Committed**
    - 트랜잭션이 커밋되어 확정된 데이터만 다른 트랜잭션이 읽도록 허용함으로써 Dirty Read를 방지해줌.
    - 커밋된 데이터만 읽더라도 Non-Repeatable Read와 Phantom Read 현상을 막지는 못함.
    - Non-Repeatable Read, Phantom Read
- **Repeatable Read**
    - 트랜잭션 내에서 쿼리를 두 번 이상 수행할 때, 첫 번째 쿼리에 있던 레코드가 사라지거나 값이 바뀌는 현상을 방지해 줌
    - 이는 트랜잭션 격리성 수준이 Phantom Read 현상을 막지는 못함.
    - Phantom Read
- **Serializable Read**
    - 트랜잭션 내에서 쿼리를 두 번 이상 수행할 때, 첫 번째 쿼리에 있던 레코드가 사라지거나 값이 바뀌지 않음은 물론 새로운 레코드가  
      나타나지도 않음

- 모든 DBMS가 4가지 레벨을 다 지원하지 않음
- SQL Server와 DB2는 4가지 레벨을 다 지원하지만 오라클은 Read Committed와 Serializable Read만 지원
- 대부분의 DBMS는 Read Committed를 기본 트랜잭션 격리성 수준으로 사용
- 다중 트랜잭션 환경에서 DBMS가 제공하는 기능을 이용해 동시성을 제어하려면 트랜잭션 시작 전에 명시적으로  
  SET TRANSACTION 명령어를 수행하면 됨
- 트랜잭션 격리성 수준을 Repeatable Read나 Serializable Read로 올리면 ISO에서 정한 기준을 만족해야 하며,  
  대부분 DBMS가 이를 구현하기 위해 Locking 매커니즘에 의존한다. 

# 제 3절. 동시성 제어

- DBMS는 다수의 사용자를 가정하며, 동시에 작동하는 다중 트랜잭션의 상호 간섭 작용에서 데이터베이스를 보호 할 수 있어야 하며, 이를 동시성 제어(Concurrency Control)이라 한다.
- 동시성을 제어할 수 있도록 하기 위해 모든 DBMS가 공통적으로 Lock 기능을 제공
- SET TRANSACTION 명령어를 이용해 트랜잭션 격리성 수준을 조정할 수 있는 기능도 제공.
- SQL Server의 경우, 기본 트랜잭션 격리성 수준인 Read committed 상태에선 레코드를 읽고 다음 레코드로 이동하자 마자 공유 Lock을 해제하지만,   Repeatable Read로 올리면 트랜잭션을 커밋될 때까지 공유 Lock을 유지
- 동시성과 일관성의 상관관계

## 1. 비관적 동시성 제어 vs. 낙관적 동시성 제어

### 가. 비관적 동시성 제어 (Pessimistic concurrency Contorl)
- 사용자들이 같은 데이터를 동시에 수정할 것이라고 가정
- 데이터를 읽는 시점에 Lock을 걸고 트랜잭션이 완료될 때까지 이를 유지
- select 시점에 Lock을 거는 비관적 동시성 제어는 자칫 시스템 동시성을 심각하게 떨어뜨릴 우려가 있음
- 아래와 같이 wait 또는 nowait 옵션을 함께 사용하는 것이 바람직

### 나. 낙관적 동시성 제어(Optimistic concurrency control)
- 사용자들이 같은 데이터를 동시에 수정하지 않을 것이라고 가정
- 이런 이유로 데이터를 읽을 때는 Lock을 설정하지 않음
- 대신 수정 시점에, 다른 사용자에 의해 값이 변경됐는지를 반드시 검사해야 함

## 2. 다중버전 동시성 제어

### 가. 일반적인 Locking 메커니즘의 문제점
- 동시성 제어의 목표는 동시에 실행되는 트랜잭션 수를 최대화하면서도 입력, 수정, 삭제 ,검색 시 데이터 무결성이 유지하는데 있다. 
- 읽기 작업에 공유 Lock을 사용하는 일반적인 Locking 매커니즘에서는 읽기 작업과 쓰기 작업이 서로 방해를 일으키기 때문에 종종 동시성에 문제가 발생
- 데이터를 일관성에 문제가 생기는 경우도 있어 이를 해결하려면 Lock을 더 오랫동안 유지하거나 테이블 레벨 Lock을 사용해야 하므로 동시성 저하 발생
- 비일관성 읽기 문제를 해결하기 위한 일반적인 해법은 트랜잭션 격리성 수준을 상향  
  기본 트랜잭션 격리성 수준(Read comitted)에서는 값을 읽는 순간에만 공유 Lock을 걸었다가 다음 레코드로 이동할 떄 Lock을 해제함으로써 위와 같은 현상이 발생
- 트랜잭션 격리성 수준을 Repeatable Read로 올리면 TX1 쿼리가 진행되는 동안 읽은 레코드는 공유 Lock이 계속 유지되며 심지어 쿼리가 끝나고 다음 쿼리가 진행되는 동안에도 유지된다.
- 트랜잭션 격리성 수준을 올리면 일관성이 높아지지만, Lock이 더 오래 유지됨으로 인해 동시성 저하 시키고 교착상태가 발생할 가능성도 커짐

### 나. 다중버전 동시성 제어
- ORACLE은 버전 3부터 다중버전 동시성 제어(Multiversion Concurrency Control, MVCC) 매커니즘을 사용
- MS SQL Server 2005, IBM DB2 9.7버전 부터 동시성 매커니즘을 제공하기 시작
- MVCC란?
    - 데이터를 변경할 때마다 그 변경사항을 UNDO 영역에 저장
    - 데이터를 읽다가 쿼리(또는 트랜잭션)시작 시점 이후에 변경된(변경이 진행중이거나 이미 커밋된)값을 발견하면, UNDO 영역에 저장된 정보를 이용해 쿼리(또는 트랜잭션)시작 시점의 일관성 있는 버전(CR Copy)를 생성하고 읽음
    - 쿼리 도중 배타적 Lock이 걸린, 즉 변경이 진행 중인 레코드를 만나더라도 대기하지 않기 때문에 동시성 측면에 유리
    - UNDO 블록 I/O, CR Copy 생성, CR 블록 캐싱 같은 부가적인 작업의 오버헤드 발생
    - Oracle은 UNDO 데이터를 UNDO 세그먼트에 저장 혹, SQL Server는 tempdb에 저장
    - MVCC는 문자수중과 트랜잭션 수준의 읽기 일관성이 존재

### 다. 문장수준 읽기 일관성
- 다른 트랜잭션에 의해 데이터의 추가, 변경, 삭제가 발생하더라도 단일 SQL문 내에서 일관성 있게 값을 읽는것을 말함.
- 위의 그림은 10023 시점에서 시작된 쿼리가 10023 시점 이후에 변경된 데이터 블록을 만났을 때,   
  Rollback(=UNDO) 세그먼트에 저장된 정보를 이용해 10023 이전 시점으로 되돌리고서 값을 읽는 것을 표현
- SQL Server에서 문장수준 읽기 일관성 모드로 DB를 운영하려면 아래 명령어를 수행.

### 라. 트랜잭션 수준 읽기 일관성
- 트랜잭션 수준 읽기 일관성은, 다른 트랜잭션에 의해 데이터의 추가, 변경 삭제가 발생하더라도 트랜잭션 내에서 일관성 있게 값을 읽는 것
- Read committed에서 완벽한 문장수준의 읽기 일관성을 보장하는 MVCC 매커니즘도 트랜잭션 수준의 읽기 일관성은 보장하지 않음
- 일반적인 Locking 매커니즘도 트랜잭션 수준의 읽기 일관성은 보장하지 않음
- 트랜잭션 수준으로 완벽한 읽기 일관성을 보장받으려면 격리성 수준을 Serializable Read로 올려주어야 함.
- Isolation Level을 Serializable Read로 상향조정하면, 일관성 기준 시점은 트랜잭션 시작 시점이 된다.  
  물론 트랜잭션이 진행되는 동안 자신이 발생시킨 변경사항은 그대로 읽음
- UNDO 데이터를 활용함으로써 높은 수준의 동시성과 읽기 일관성을 유지하는 대신, 일반적인 Locking 매커니즘에 없는 SNAPSHOT TOO OLD 에러가 MVCC에서 발생
- UNDO 영역에 저장된 UNDO 정보가 다른 트랜잭션에 의해 재사용돼 필요한 CR Copy을 생성할 수 없을 때 발생

- SNAPSHOT TOO OLD 발생 가능성을 줄이는 방법
    - UNDO 영역의 크기를 증가
    - 불필요한 커밋을 자주 수행하지 않음
    - FETCH ACROSS COMMIT 형태의 프로그램 작성을 피해 다른 방식으로 구현
    - ANSI 표준에 따르면 커밋 이전에 열려 있던 커서는 더는 FETCH 하면 안됨
    - 트랜잭션이 몰리는 시간대에 오래 걸리는 쿼리가 같이 수행되지 않도록 조정
    - 큰 테이블을 일정 범위로 나누어 읽고 단계적으로 실행할 수 있도록 코딩
    - SNAPSHOT TOO OLE 발생 가능성을 줄일 뿐 아니라 문제가 발생시 특정 부분부터 다시 시작할 수 있음 -> 읽기 일관성에 문제가 없을때만 적용
    - 오랜 시간에 걸쳐 같은 블록을 여러번 방문하난 NL Join 형태의 조인문 또는 인덱스를 경유한 테이블 액세스를 수반하는 프로그램이 있는지 체크
    - 이를 회피할 수 있는 방법(조인 메소드 변경, FUll Table Scan등)을 찾음
    - 소트 부하를 감수하더라도 order by 등을 강제로 삽입해 소트 연산이 발생하도록 함 
    - 대량 업데이트 후에 곧바로 해당 테이블 또는 인덱스를 Full Scan 하도록 쿼리를 수행하는 것도 하나의 해결 방법

ㅇㄴㅇㄴ